In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import pandas as pd

In [2]:
import firebase_admin
from firebase_admin import credentials, db

# Firebase가 이미 초기화되었는지 확인 후 실행
if not firebase_admin._apps:
    cred = credentials.Certificate(r"C:\Users\302-15\Desktop\새 폴더 (2)\myproject-a52d3-firebase-adminsdk-fbsvc-4c7091eb33.json")  # JSON 파일 경로 확인
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://myproject-a52d3-default-rtdb.asia-southeast1.firebasedatabase.app/'
    })

# Firestore 또는 Realtime Database 사용
ref = db.reference('/')
print("✅ Firebase 연결 성공!")


✅ Firebase 연결 성공!


In [3]:

# 🔥 1. Firebase 서비스 계정 키 파일 경로 (올바르게 입력)
cred = credentials.Certificate(r"C:\Users\302-15\Desktop\새 폴더 (2)\myproject-a52d3-firebase-adminsdk-fbsvc-4c7091eb33.json")

# 🔥 2. Firebase가 이미 초기화되었는지 확인 후 실행
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://myproject-a52d3-default-rtdb.asia-southeast1.firebasedatabase.app/'
    })

# 🔥 3. 데이터베이스 참조
dbRef = db.reference('/')  # 루트 경로

ref.set({
    'users': {
        'user1': {
            'name': '홍길동',
            'age': 25
        },
        'user2': {
            'name': '김철수',
            'age': 30
        }
    }
})

# 🔥 4. 데이터 가져오기
data = dbRef.get()
print("✅ Firebase 데이터:", data)


✅ Firebase 데이터: {'users': {'user1': {'age': 25, 'name': '홍길동'}, 'user2': {'age': 30, 'name': '김철수'}}}


In [6]:
import firebase_admin
from firebase_admin import credentials, db
import numpy as np
import pandas as pd
from datetime import datetime

# 🔥 Firebase 서비스 계정 키 파일 경로 입력
cred = credentials.Certificate(r"C:\Users\302-15\Desktop\새 폴더 (2)\myproject-a52d3-firebase-adminsdk-fbsvc-4c7091eb33.json")

# 🔥 Firebase 중복 초기화 방지
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://myproject-a52d3-default-rtdb.asia-southeast1.firebasedatabase.app/'
    })

# 🔥 Firebase 데이터베이스 참조
dbRef = db.reference()

# 🔥 CSV 파일 로드
csv_file_path = "OTg6QzA_activities.csv"
act = pd.read_csv(csv_file_path)

# 🔥 timestamp 컬럼 존재 여부 확인
if 'timestamp' not in act.columns:
    raise KeyError("🚨 'timestamp' 컬럼이 존재하지 않습니다. CSV 파일의 컬럼명을 확인하세요!")

# 2022년 1월 1일 데이터만 추출
data220101 = act[(act['timestamp'] >= '2022-01-01 00:00:00.000') & 
                 (act['timestamp'] < '2022-01-02 00:00:00.000')].copy()

data220101['p_timestamp'] = data220101.timestamp.shift(1)
data220101['p_x'] = data220101['X'].shift(1)
data220101['p_y'] = data220101['Y'].shift(1)
data220101['p_z'] = data220101['Z'].shift(1)

# NaN 값 제거
tenmin = data220101.dropna().copy()

# 거리 계산 함수
def calc_dist(df):
    x_d = df['X'] - df['p_x']
    y_d = df['Y'] - df['p_y']
    z_d = df['Z'] - df['p_z']
    return np.sqrt(x_d**2 + y_d**2 + z_d**2)

# 시간 차이 계산 함수
def calc_timediff(df):
    d1 = datetime.strptime(str(df['timestamp']), "%Y-%m-%d %H:%M:%S.%f")
    d2 = datetime.strptime(str(df['p_timestamp']), "%Y-%m-%d %H:%M:%S.%f")
    return (d1 - d2).total_seconds()

# 속도 계산 함수
def calc_act(df):
    if df['d_time'] != 0:
        return np.abs(df['dist'] / df['d_time'])
    else:
        return 0

tenmin['dist'] = tenmin.apply(calc_dist, axis=1)
tenmin['d_time'] = tenmin.apply(calc_timediff, axis=1)
tenmin['calc_act'] = tenmin.apply(calc_act, axis=1)
tenmin['timestamp'] = tenmin['timestamp'].astype('datetime64[ns]')

cleaned = tenmin[['timestamp', 'calc_act']].set_index('timestamp')

# 10분 단위 리샘플링
result = cleaned.resample('10min').agg(['mean'])
result.columns = ['mean']

# 🔥 칼럼 이름 변경 및 시간 형식 변환
result['hm'] = result.index.to_series().dt.strftime('%H:%M')

# 🔥 NaN 값을 0으로 채우기
result.fillna(0, inplace=True)

# 🔥 Firebase에 업로드할 데이터 변환
updates = result.to_dict(orient='records')
print("✅ 변환된 데이터 형식:", updates[:5])
# 🔥 Firebase에 데이터 저장
dbDevice = dbRef.child('OTg6QzA')
dbDevice.child('2022-01-01').set(updates)

print("✅ Firebase 데이터 업로드 완료!")


✅ 변환된 데이터 형식: [{'mean': 2.359774251549589, 'hm': '00:00'}, {'mean': 3.8049098618070056, 'hm': '00:10'}, {'mean': 1.2044129884418076, 'hm': '00:20'}, {'mean': 0.06560836089278393, 'hm': '00:30'}, {'mean': 0.0, 'hm': '00:40'}]
✅ Firebase 데이터 업로드 완료!
